# Grid Cells

The goal of this notebook is to understand the topology of the population activity of a module of grid cells.

<video controls width="400" src="../figures/population_experiments.mp4" />

- Gardner, R.J., Hermansen, E., Pachitariu, M. et al. _Toroidal topology of population activity in grid cells._ **Nature** 602, 123–128 (2022).

**Grid cells** are neurons present in rats and other mammals (including hummans) that constitute a positioning system in the brain, encoding a neural representation of a metric in the 2-dimensional space. The arrangement of each grid cell's spatial firing fields, all at equal distances from their neighbours, forms an hexagonal pattern as discovered by Edvard Moser and May-Britt Moser, who were awarded the 2014 Nobel Prize in Physiology or Medicine for their work.

<video controls src="../figures/grid_cells.mp4" />

## 1. The point cloud

**Download the data from https://figshare.com/articles/dataset/Toroidal_topology_of_population_activity_in_grid_cells/16764508**

**Load the file 'rat_r_day1_grid_modules_1_2_3.npz' in a variable *data* (we will only focus on *Rat R Day 1*).**

In [53]:
data = 

* **Question:** What's the data available in this file?

## 2. Position

We select the interval of the session when the rat is moving in an open field, that is, a square track (this information is available in the file 'rat_r_day1_sessions.txt').

In [ ]:
#open_field_1, start=7457, end=16045, valid_times=[7457,14778;14890,16045]
#Here we compute the intervals of time at which the rat was moving in open field.
start1= 7457.01
end1  = 14778.00
tmin1 = int((start1-data['t'].min())*100)
tmax1 = int((end1-data['t'].min())*100)
start2= 14890.01
end2  = 16045.00
tmin2 = int((start2-data['t'].min())*100)
tmax2 = int((end2-data['t'].min())*100)

**Create a variable *positions* and restrict the positions _data['x']_ and _data['y']_ to the intervals _[tmin1, tmax1]_ and _[tmin2, tmax2]_**

In [4]:
positions =

**Plot the path of the rat in the square track.**

## 3. Firing rates (module 1)

We restrict to the firing activity of the neurons at module 1.

In [ ]:
mod = data['spikes_mod1'].item() #this variable encodes, for every cell, the list of times of the spikes

We first compute (linearly infer) the position of the rat at the firing times.

In [6]:
def linear_combination(data_pos, data_spikes, index0, t0, delta_t):
    '''
    Computes (as a linear inference) the position of the animal at the spike times (data_spikes) from the data of its position (data_pos)
    at equidistant intervals of time (delta_t)
    '''
    n = len(index0)
    index1 = [int(x) for x in index0 + np.ones(n)]
    v = (data_pos[index1]-data_pos[index0])/delta_t * (data_spikes-t0) + data_pos[index0]
    return v

pos = {} #this variable will encode, for every cell, the list of positions associated to the spikes
tmin = data['t'].min() #the starting time

for cell, spikes in mod.items():
    spikes = mod[cell]
    openfield = spikes[np.logical_or((spikes<end1) & (spikes>start1), (spikes<end2) & (spikes>start2))]
    nspikes = len(openfield)
    pos[cell] = np.zeros((2, nspikes))
    t0 = np.floor(openfield*100)/100 #2 decimals
    index0 = [int(x) for x in np.floor((t0-tmin)*100)]
    pos[cell][0] = linear_combination(data['x'], openfield, index0, t0, 0.01)
    pos[cell][1] = linear_combination(data['y'], openfield, index0, t0, 0.01)

We compute now the firing rate maps

In [7]:
from scipy.ndimage import gaussian_filter

def discretize(cell_dgm, m=0, M=1, res=30, filt=None, sigma=0.5):
    '''
    Discretize one cell map, returns a 2D-histogram of size (res x res), the grid representing [m, M]^2.
    '''
    h2d = np.histogram2d(cell_dgm[0,:], cell_dgm[1,:], bins=res, range=[[m, M],[m, M]])[0]
    h2d = np.flip(h2d.T, axis=0)
    if filt=="gaussian":
        h2d = gaussian_filter(h2d, sigma=sigma)
    return h2d

n_bins = 30 # number of pixels (by side) used to discretize the square track
l0 = -0.725 # left bound
l1 = 0.725  # right bound

smooth_positions = discretize(positions.T,l0, l1, n_bins, "gaussian", 1)  # the (smooth) frequency of the animal at every pixel

fire_rate_grid = {} # encodes, for every cell, the firing rate for every pixel (over the requency of the aminal at the pixel)
for cell in pos.keys():
    fire_rate = discretize(pos[cell], l0, l1, n_bins, "gaussian", 1)
    fire_rate_grid[cell] = (fire_rate/smooth_positions).flatten()

**Plot the firing rate maps for every grid cell in module 1.**

## 3. The geometry of neural activity

In [9]:
population_data = np.array(list(fire_rate_grid.values())).T 
#every point in the point cloud encodes the firing rate 
#at a certain pixel of all the neurons. 
population_data.shape

(900, 166)

**Question:** How many pixels are in the square track? How many grid cells are in the module?

**Compute the Isomap projection on 3 dimensions of the population data.**

**Plot the embedding.**

**Question:** Can you identify the shape of the point cloud?

## 4. Persistent Homology

**Compute the geodesic distance between every pair of the points in the population data.** Use k=15

*Hint: Use the auxiliary functions from utils.py*

In [ ]:
distance_matrix = 

**Compute and plot the persistent diagram using as input the distance matrix of estimated geodesic distances**

## 5. Orientability

**Question:** Is the manifold orientable?
Compare the persistence diagrams with coefficients in $\mathbb{Z}_2$ and $\mathbb{Z}_3$. 

If you are too enthusiastic, compute the local dimension at every point and decide if there are singularities or boundary.